MS-COCO 데이터셋 다운로드 및 전처리: 이미지와 캡션 데이터를 준비합니다.
- 특징 추출 모델 (VGG): 이미지를 입력받아 특징을 추출합니다.
- 캡셔닝 모델 (Transformer): 추출된 특징을 이용해 이미지를 설명하는 문장을 생성합니다.
- 여기서는 Python과 PyTorch를 사용하여 VGG 모델로 특징을 추출하고, Transformer 모델을 사용해 캡션을 생성하는 과정을 단계별로 구현해보겠습니다.

1. 데이터셋 다운로드 및 전처리
MS-COCO 데이터셋은 매우 크기 때문에 부분만 사용하는 것이 좋습니다. 먼저, 데이터셋을 다운로드하고 전처리하는 코드를 작성합니다.

In [ ]:

import os
import json
import torch
from torch.utils.data import Dataset, DataLoader
from PIL import Image
from torchvision import transforms
from pycocotools.coco import COCO

# 데이터셋 경로 설정
data_dir = 'path_to_coco_dataset'
train_dir = os.path.join(data_dir, 'train2017')
ann_file = os.path.join(data_dir, 'annotations', 'captions_train2017.json')

# 이미지 전처리
transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize((0.485, 0.456, 0.406), (0.229, 0.224, 0.225))
])

# COCO 데이터셋 로드
coco = COCO(ann_file)

# 데이터셋 클래스 정의
class CocoDataset(Dataset):
    def __init__(self, root, annFile, transform=None):
        self.root = root
        self.coco = COCO(annFile)
        self.ids = list(self.coco.anns.keys())
        self.transform = transform

    def __getitem__(self, index):
        ann_id = self.ids[index]
        caption = self.coco.anns[ann_id]['caption']
        img_id = self.coco.anns[ann_id]['image_id']
        path = self.coco.loadImgs(img_id)[0]['file_name']
        
        img = Image.open(os.path.join(self.root, path)).convert('RGB')
        if self.transform is not None:
            img = self.transform(img)
        
        return img, caption

    def __len__(self):
        return len(self.ids)


In [ ]:
# 데이터셋 인스턴스 생성
dataset = CocoDataset(root=train_dir, annFile=ann_file, transform=transform)
dataloader = DataLoader(dataset, batch_size=32, shuffle=True, num_workers=4)

2. VGG 모델을 이용한 특징 추출
VGG 모델을 이용해 이미지에서 특징을 추출하는 코드를 작성합니다. 여기서는 torchvision 라이브러리의 미리 학습된 VGG 모델을 사용합니다.

In [ ]:
from torchvision import models

# 미리 학습된 VGG 모델 로드
vgg = models.vgg16(pretrained=True)
vgg.classifier = torch.nn.Sequential(*list(vgg.classifier.children())[:-1])  # 마지막 FC layer 제거
vgg.eval()

def extract_features(image):
    with torch.no_grad():
        features = vgg(image)
    return features


3. Transformer 모델 정의 및 학습
이제 Transformer 모델을 정의하고 학습하는 코드를 작성합니다. 여기서는 PyTorch의 nn.Transformer를 사용합니다.

In [ ]:
import torch.nn as nn

class ImageCaptioningModel(nn.Module):
    def __init__(self, feature_dim, vocab_size, embed_size, num_heads, num_layers):
        super(ImageCaptioningModel, self).__init__()
        self.feature_embed = nn.Linear(feature_dim, embed_size)
        self.transformer = nn.Transformer(d_model=embed_size, nhead=num_heads, num_encoder_layers=num_layers, num_decoder_layers=num_layers)
        self.fc_out = nn.Linear(embed_size, vocab_size)
        self.embed = nn.Embedding(vocab_size, embed_size)
        self.positional_encoding = nn.Parameter(torch.zeros(1, 1000, embed_size))

    def forward(self, features, captions):
        features = self.feature_embed(features)
        features = features.unsqueeze(1)
        captions = self.embed(captions)
        captions = captions + self.positional_encoding[:, :captions.size(1), :]
        
        transformer_out = self.transformer(features.permute(1, 0, 2), captions.permute(1, 0, 2))
        output = self.fc_out(transformer_out)
        
        return output

# 하이퍼파라미터 설정
vocab_size = 10000  # 예시 값, 실제로는 데이터셋에 맞게 설정해야 함
embed_size = 512
num_heads = 8
num_layers = 6
feature_dim = 4096  # VGG16의 출력 차원

# 모델 인스턴스 생성
model = ImageCaptioningModel(feature_dim, vocab_size, embed_size, num_heads, num_layers)

# 손실 함수와 옵티마이저 정의
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)

# 학습 루프
num_epochs = 10
for epoch in range(num_epochs):
    model.train()
    for images, captions in dataloader:
        features = extract_features(images)
        optimizer.zero_grad()
        outputs = model(features, captions[:, :-1])
        loss = criterion(outputs.view(-1, vocab_size), captions[:, 1:].contiguous().view(-1))
        loss.backward()
        optimizer.step()
    
    print(f'Epoch [{epoch+1}/{num_epochs}], Loss: {loss.item():.4f}')
